In [12]:
from tqdm import tqdm

import torch
from torch import mps
from torch import nn
from torch.utils.data import DataLoader, random_split

from torchvision import transforms, datasets
from sklearn.metrics import accuracy_score

import PIL

In [14]:
IMAGE_SIZE = 320

EPOCHES = 5
BATCH_SIZE = 64
LEARNING_RATE = 0.001

PATH = 'New_dataset'

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),    
            
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
                
            nn.Flatten(),
            nn.Linear(16384, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(500, 2))
        
    def forward(self, xb):
        return self.network(xb)

In [16]:
def random_split_ratio(dataset, test_size=.2, random_state=None):
    second_part = int(len(dataset) * test_size)
    first_part = int(len(dataset)) - second_part

    if random_state:
        first_split, second_split = random_split(dataset, lengths=[first_part, second_part], 
                                                 generator=torch._C.Generator().manual_seed(random_state))
    else:
        first_split, second_split = random_split(dataset, lengths=[first_part, second_part])

    return first_split, second_split

In [17]:
def verify_image(fp):
    try:
        PIL.Image.open(fp).verify()
        return True
    except:
        return False

In [18]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f'CNN training on {device}')

CNN training on mps


In [19]:
model = CNN()

In [20]:
transform = transforms.Compose([
    transforms.Resize(size=100),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=120),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor()
])

In [21]:
dataset = datasets.ImageFolder(root=PATH, transform=transform, is_valid_file=verify_image)

dataset, tests_dataset = random_split_ratio(dataset, random_state=42, test_size=.22)
tests_loader = DataLoader(tests_dataset, batch_size=BATCH_SIZE, shuffle=True)

train_dataset, valid_dataset = random_split_ratio(dataset, random_state=42, test_size=.3)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f'Train data length: {len(train_loader.dataset)}\n'
      f'Valid data length: {len(valid_loader.dataset)}\n'
      f'Tests data length: {len(tests_loader.dataset)}')

Train data length: 16708
Valid data length: 7160
Tests data length: 6732


In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [23]:
for epoch in range(EPOCHES):
    torch.mps.empty_cache()
    torch._C._mps_emptyCache()
    model.to(device)
    
    train_running_loss = 0
    valid_running_loss = 0
    bar = tqdm(train_loader, total=len(train_loader), ncols=100)
    bar.set_description(f"Training    ")
    
    for x_batch, y_batch in bar:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad()

        train_running_loss += (loss.item() * x_batch.size(0))

    with torch.no_grad():  
        model.eval()

        valid_predict = []  
        valid_targets = []  
        
        vbar = tqdm(valid_loader, total=len(valid_loader), ncols=100)
        vbar.set_description(f"Validation  ")
        vbar.reset()
        
        for x_batch, y_batch in vbar:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            
            valid_running_loss += (loss.item() * x_batch.size(0))

            valid_predict.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            valid_targets.extend(y_batch.cpu().numpy())

        model.train()

    print(f'Epoch:               {(epoch + 1):2d} / {EPOCHES}\n'
          f'Avarage train loss:  {(train_running_loss / len(train_loader.dataset)):.3f}\n'
          f'Avarage valid loss:  {(valid_running_loss / len(valid_loader.dataset)):.3f}\n'
          f'Validation accuracy: {(accuracy_score(valid_targets, valid_predict)):.3f}')

Training    :  16%|███████▉                                        | 43/262 [00:25<02:08,  1.71it/s]


KeyboardInterrupt: 

In [16]:
model.eval()

test_predict = []  
test_targets = []  

bar = tqdm(enumerate(tests_loader), total=len(tests_loader), ncols=100)
bar.set_description(f"Testing     ")

for idx_batch, (x_batch, y_batch) in bar:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    outputs = model(x_batch)
    test_predict.extend(torch.argmax(outputs, dim=1).cpu().numpy())
    test_targets.extend(y_batch.cpu().numpy())
    
accuracy = accuracy_score(test_targets, test_predict)
print(f'Test Accuracy: {100 * accuracy:.2f}%')

Testing     : 100%|███████████████████████████████████████████████| 106/106 [00:18<00:00,  5.61it/s]

Test Accuracy: 99.51%


In [13]:
torch.save(model.state_dict(), str('clothing classifier ' + str(round(accuracy * 100, 2)) + ' persent.pth'))

In [15]:
persent = 99.53
model.load_state_dict(torch.load(str('clothing classifier ' + str(persent) + ' persent.pth')))

<All keys matched successfully>